**Дополнительное домашнее задание 1 - 10 баллов**

1. Загрузите набор данных lenta-ru-news с помощью библиотеки Corus для задачи классификации текстов по топикам
2. Опишите, насколько, по вашему мнению, данные требуют предобработки для задачи тематического моделирования. При необходимости, проведите релевантную предобработку текстов. **1 балл**
3. Используйте библиотеку BERTopic для тематического моделирования:
    - Подберите оптимальные, на ваш взгляд, элементы пайплайна: энкодер, снижение размерности, алгоритм кластеризации, способ токенизации, постобработку/тюнинг.
При выборе инструмента на каждом шаге опишите, почему был выбран именно он среди многочисленных альтернатив. **2 балла**
    - Настройте ваш пайплайн, подобрав оптимальные гиперпараметры для отдельных шагов. При выборе значений конкретных гиперпараметров укажите, почему остановились на тех или иных значениях. **1 балл**
4. Визуализируйте полученные результаты: **2 балла**
    - Топ-токены для каждого топика.
    - Документы с их топиками в 2D пространстве
    - Распределение тем по токенам для выборочных текстов из датасета
5. Оцените формальное качество лучшего результата с помощью метрик для тематического моделирования:  **2 балла**
    - Topic Diversity
    - UMass Coherence
Hint: реализацию можно написать самостоятельно или поискать в таких библиотеках, как [Gensim](https://github.com/piskvorky/gensim) и [OSTIS](https://github.com/MIND-Lab/OCTIS)
6. Проанализируйте полученные результаты в совокупности и резюмируйте, что удалось, какие проблемы вы заметили, как их можно решить в дальнейшем. **1 балл**

**Задание выполнялось в Google Colab**

In [ ]:
!pip install corus

In [ ]:
!pip install natasha

In [ ]:
!pip install bertopic

In [ ]:
import nltk
import pandas as pd
import numpy as np
import math
import re
import spacy
import nltk
import random
from corus import load_lenta

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
from nltk.corpus import stopwords
from tqdm.auto import tqdm
import urllib.request
from hdbscan import HDBSCAN

from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction

from collections import defaultdict

In [ ]:
# Обеспечим воспроизводимость ноутбука
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

**1. Загрузка и предобработка данных**

Чтобы улучшить качество тематического моделирования необходимо выполнить стандартную предобработку текста:
- удалить знаки препинания (не несут смысловой нагрузки)
- токенизировать текст (разделим текст на слова)
- удалить стоп-слова (избежим влияния частых и неинформативных слов)
- лемматизация (чтобы темы не зависели от формы слова)

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

print(next(records))

LentaRecord(url='https://lenta.ru/news/2018/12/14/cancer/', title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака', text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.', topic='Россия', tags='Общество', date=None)


In [ ]:
df = pd.DataFrame(((r.text) for r in records), columns=['text'])
df.head()

,text
0,Австрийские правоохранительные органы не предс...
1,Сотрудники социальной сети Instagram проанализ...
2,С начала расследования российского вмешательст...
3,Хакерская группировка Anonymous опубликовала н...
4,Архиепископ канонической Украинской православн...


In [ ]:
# Насэмплируем случайным образом 10_000 текстов, чтобы
df_sample = df.sample(n=10000, random_state=42)

In [ ]:
nltk.download('stopwords')

In [ ]:
# Список русских стоп-слов
russian_stopwords = set(stopwords.words("russian"))

def remove_stopwords(text):
    # Убираем пунктуацию и приводим к нижнему регистру
    words = re.findall(r'\w+', text.lower())
    filtered = [word for word in words if word not in russian_stopwords]
    return ' '.join(filtered)

tqdm.pandas(desc="Delete stop-words")

# Применим к нашим данным
df_sample['cleaned'] = df_sample['text'].progress_apply(remove_stopwords)

Delete stop-words:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
df_sample.head()

,text,cleaned
153198,Группа ученых из Принстонского университета и ...,группа ученых принстонского университета униве...
169154,В Северо-Кавказском окружном военном суде выне...,северо кавказском окружном военном суде вынесл...
83745,Президент России Владимир Путин поздравил евре...,президент россии владимир путин поздравил евре...
10029,Американская супермодель и участница реалити-ш...,американская супермодель участница реалити шоу...
6445,Житель британского города Уэстклифф-он-Си (гра...,житель британского города уэстклифф си графств...


In [ ]:
# Инициализируем модели Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [ ]:
# Функция лемматизации
def lemmatize_text(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmas = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemmas.append(token.lemma)

    return ' '.join(lemmas)

tqdm.pandas(desc="Lemmatization")

# Применим к нашим данным
df_sample['lemmatized'] = df_sample['cleaned'].progress_apply(lemmatize_text)

Lemmatization:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
df_sample.head()

,text,cleaned,lemmatized
153198,Группа ученых из Принстонского университета и ...,группа ученых принстонского университета униве...,группа ученый принстонский университет универс...
169154,В Северо-Кавказском окружном военном суде выне...,северо кавказском окружном военном суде вынесл...,северо кавказский окружный военный суд вынести...
83745,Президент России Владимир Путин поздравил евре...,президент россии владимир путин поздравил евре...,президент россия владимир путин поздравить евр...
10029,Американская супермодель и участница реалити-ш...,американская супермодель участница реалити шоу...,американский супермодель участница реалить шоу...
6445,Житель британского города Уэстклифф-он-Си (гра...,житель британского города уэстклифф си графств...,житель британский город уэстклифф си графство ...


**2. BERTopic**

**2.1 Модель эмбеддингов:**

Модель основана на cointegrated/LaBSE-en-ru - имеет аналогичные размеры контекста (512), эмбеддингов (768) и достаточно быстрая. Думаю она неплохо подойдет под условия нашей задачи.

In [ ]:
embedding_model = SentenceTransformer("sergeyzh/LaBSE-ru-turbo")

**2.2 DIM модель:**

Обычно для снижения размерности эмбеддингов перед кластеризацией в **BERTopic** используется **UMAP** . Однако **TruncatedSVD** — более стабильный и лучше работает с разреженными матрицами (например, TF-IDF), а также быстрее и воспроизводимее, особенно для большого объёма текстов, поэтому будем использовать его.

In [ ]:
dim_model = TruncatedSVD(
    n_components=300,  # Подбираем по explained variance
    n_iter=10,  # Лучше сходимость
    random_state=42,  # воспроизводимость
)

**2.3 Cluster модель:**

**HDBSCAN** — продвинутый алгоритм кластеризации, способный определять кластеры различной плотности и выделять шум (неприменимые тексты). Это важно при работе с новостными текстами, где могут встречаться выбросы или разнородные темы.

In [ ]:
cluster_model = HDBSCAN(
    min_cluster_size=5,
    min_samples=1,
    metric='euclidean',
    cluster_selection_method='leaf',
    prediction_data=True,
)


**2.4 Векторизация:**

Простой и эффективный способ извлечения частотных признаков из текста. TF-IDF придаёт больший вес словам, специфичным для темы, что важно для выделения ключевых слов темы. Это основной способ перевода текста в вектор перед применением c-TF-IDF.

In [ ]:
vectorizer_model = TfidfVectorizer()

**2.5 с-TF-IDF:**

Class-based TF-IDF (c-TF-IDF) — особая адаптация TF-IDF, в которой тексты одного кластера обрабатываются как единый документ. Это усиливает тематические различия между кластерами.

In [ ]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

**2.6 Representation model:**

Используется для переопределения ключевых слов, описывающих тему, на основе их семантической значимости (а не только частоты). KeyBERTInspired пересчитывает наиболее репрезентативные ключевые слова с учётом контекста. Это особенно полезно для повышения качества интерпретации тем, особенно на русском, где ключевые слова могут быть многозначны.

In [ ]:
representation_model = KeyBERTInspired()

In [ ]:
# Собираем нашу модель из загруженных компонентов
topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=dim_model,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
)

In [ ]:
#Обучаем модель

%%time

topics, probs = topic_model.fit_transform(df_sample['lemmatized'])

CPU times: user 4min 41s, sys: 1.22 s, total: 4min 42s
Wall time: 4min 35s


**3. Визуализация**

**Визуализируем топ-10 тем**

In [ ]:
topic_model.get_topic_info()[1:11]

,Topic,Count,Name,Representation,Representative_Docs
1,0,52,0_студийный_scars_пластинка_лейбл,"[студийный, scars, пластинка, лейбл, сольный, ...",[группа оззи осборн black sabbath появиться за...
2,1,37,1_playstation_геймер_eurogamer_fallout,"[playstation, геймер, eurogamer, fallout, kill...",[студия platinumgames заинтересовать создание ...
3,2,33,2_украинец_боксерский_весовой_кличко,"[украинец, боксерский, весовой, кличко, boxing...",[бывший чемпион мир бокс тяжелый вес американе...
4,3,33,3_ходорковский_лжепредпринимательство_ходатайс...,"[ходорковский, лжепредпринимательство, ходатай...",[отношение михаил ходорковский платон лебедев ...
5,4,28,4_звездообразование_астрономический_астрофизик...,"[звездообразование, астрономический, астрофизи...",[астроном удаться обнаружить островок спокойст...
6,5,24,5_сербский_югославский_серб_черногорский,"[сербский, югославский, серб, черногорский, ба...",[социалистический партия сербия войти коалиция...
7,6,24,6_котировка_понизиться_фьючерс_биржа,"[котировка, понизиться, фьючерс, биржа, фондов...",[российский фондовый индекс 9 август прервать ...
8,7,23,7_космический_космодром_аккумуляторный_орбитал...,"[космический, космодром, аккумуляторный, орбит...",[новый израильский аэробаллистический ракета r...
9,8,22,8_донецкий_ополченец_украиной_силовик,"[донецкий, ополченец, украиной, силовик, минск...",[украинский силовик усилить свой позиция горло...
10,9,22,9_генетический_генетик_геном_neanderthalensis,"[генетический, генетик, геном, neanderthalensi...",[современный человек мигрант африка победить е...


**Посмотрим на самые популяные слова в топ-30 тем**

In [ ]:
topic_model.visualize_barchart(top_n_topics=30, n_words=10, title='Топ слов по темам')

**Посмотрим на тему `[1]` в документе `[1]`**

In [ ]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(df_sample['lemmatized'], calculate_tokens=True, batch_size=100)

In [ ]:
df_example = topic_model.visualize_approximate_distribution(df_sample['lemmatized'].iloc[1], topic_token_distr[1])
df_example

,северо,кавказский,окружный,военный,суд,вынести,обвинительный,приговор,два,уроженец,дагестан,магомеджамил,лежбединов,дашир,хакимов,убийство,начальник,районный,отдел,фсб,россия,сообщать,интерфакс,лежбедин,приговорить,25,год,заключение,колония,строгий,режим,дашир,хаким,21,кроме,убийство,подсудимый,зависимость,роль,каждый,признать,виновный,бандитизме,теракт,незаконный,оборот,оружие,умышленный,уничтожение,имущество,крупный,размер,участие,незаконный,вооруженный,формирование,действовать,территория,карабудахкентский,район,республика,данные,следствие,2010,год,лежбединовый,хаким,убить,руководитель,карабудахкентский,отдел,фсб,магомедсаид,гаджиев,счет,также,покушение,жизнь,начальник,районный,контрольный,счетный,палата,абдулгамида,айдиева,21,ноябрь,сообщаться,дагестане,неизвестный,застрелить,глава,село,телетль,шамильский,район,данный,момент,вестись,поиск,нападать,глава,республика,рамазан,абдулатипов,17,ноябрь,заявить,правоохранительный,орган,удаться,стабилизировать,ситуация,регион,слово,течение,полтора,год,дагестане,допустить,один,теракт,бандподполье,территория,республика,полагать,абдулатипов,остаться,порядка,30,40,человек
97_гособвинение_москворецкий_эмират_немцов,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.103,0.212,0.212,0.212,0.108,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
105_магарамкентский_карабудахкентский_гаджиев_экстремист,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.138,0.275,0.413,0.553,0.415,0.279,0.140,0.000,0.000,0.000,0.000,0.000,0.120,0.234,0.234,0.234,0.114,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
123_хаджикурбанов_оперуполномоченный_присяжный_политковский,0.000,0.000,0.122,0.122,0.122,0.122,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.110,0.110,0.110,0.110,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
233_политковский_великобританию_отравить_иронизировать,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.103,0.103,0.103,0.103,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00

**Посмотрим в 2D**

In [ ]:
# Проведем визуализацию распределения тем
df_2d = topic_model.visualize_topics()  # Визуализируем темы
df_2d.show()

**4. Расчет метрик**

**Оценим результат по метрике Topic Diversity**

In [ ]:
def topic_diversity(topic_words, top_n=10):
    all_words = [word for topic in topic_words for word in topic[:top_n]]
    unique_words = set(all_words)
    return len(unique_words) / (top_n * len(topic_words))

topics = topic_model.get_topics()
topic_words = [words for _, words in topics.items()]
td_score = topic_diversity(topic_words, top_n=10)
print(f"Topic Diversity: {td_score:.4f}")

Topic Diversity: 1.0000


Такая высокая оценка говорит о том, что у нас отлично получилось обеспечить разнообразие топиков

**Оценим результат по метрике UMass Coherence**

In [ ]:
texts = [doc.split() for doc in df_sample['lemmatized']]

# Получаем топ-слова по темам (фильтруем пустые)
topics = topic_model.get_topics()
topic_words = [
    [word for word, _ in topic[:10]]
    for topic in topics.values()
    if len(topic) >= 10  # только темы с ≥10 словами
]
# Параметры
epsilon = 1  # сглаживание
top_n = 10   # кол-во слов в теме

# Подготовка частот
word_doc_freq = defaultdict(int)
word_pair_doc_freq = defaultdict(int)

for doc in texts:
    unique_words = set(doc)
    for word in unique_words:
        word_doc_freq[word] += 1
    for word1 in unique_words:
        for word2 in unique_words:
            if word1 != word2:
                word_pair_doc_freq[(word1, word2)] += 1

# Расчёт u_mass для каждой темы
topic_umass_scores = []

for topic in topic_words:
    score = 0.0
    pairs = 0
    for i in range(1, len(topic)):
        for j in range(i):
            wi = topic[i]
            wj = topic[j]
            D_wi_wj = word_pair_doc_freq.get((wi, wj), 0)
            D_wj = word_doc_freq.get(wj, 0)
            if D_wj > 0:
                score += math.log((D_wi_wj + epsilon) / D_wj)
                pairs += 1
    if pairs > 0:
        topic_umass_scores.append(score / pairs)

# Итоговый UMass
final_umass = sum(topic_umass_scores) / len(topic_umass_scores)
print(f"UMass Coherence: {final_umass:.4f}")

UMass Coherence: -1.6190


Отрицательное значение Umass может указывать на то, что слова внутри темы не всегда часто встречаются вместе в тех же документах. В некоторых случаях, если темы разрознены или слишком обобщены, они могут иметь отрицательное значение.

- Меньшие значения говорят о том, что для данной темы нужно улучшить её фокусировку (например, уменьшить количество пересекающихся слов из разных областей).
- Это может означать, что модель генерирует более разрозненные темы или что сам корпус не слишком однороден.

**5. Выводы**

**Чего удалось добиться:**

- Модель выделила разнообразные темы, которые можно интерпретировать (например, политика, мода, игры и строительство)

- Получены метки кластеров для документов, что позволяет анализировать распределение и выявлять основные темы

**Идеи для дальнейших улучшений:**

- Применить дополнительные методы очистки данных (например, удаление редких слов, спецсимволов)

- Поэкспериментировать с параметрами HDBSCAN (например, увеличить минимальный размер кластера)

- Попробовать другие модели тематического моделирования, такие как LDA или NMF, для сравнения результатов